In [8]:
import requests
import json
import pandas as pd
import FinanceDataReader as fdr
import statsmodels.api as sm
import seaborn as sns
from datetime import datetime

In [3]:
df_krx = fdr.StockListing('KRX')
df_krx

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,62800,2,-200,-0.32,62600,63000,62400,9382576,587897501100,374902344140000,5969782550,STK
1,373220,KR7373220003,LG에너지솔루션,KOSPI,,542000,2,-8000,-1.45,547000,548000,535000,260831,141170994000,126828000000000,234000000,STK
2,000660,KR7000660001,SK하이닉스,KOSPI,,93500,2,-900,-0.95,94300,94700,93100,1971090,185026575400,68068221127500,728002365,STK
3,207940,KR7207940008,삼성바이오로직스,KOSPI,,808000,2,-6000,-0.74,809000,811000,802000,34509,27825320000,57508592000000,71174000,STK
4,006400,KR7006400006,삼성SDI,KOSPI,,735000,2,-3000,-0.41,727000,736000,718000,198917,144721499000,50541929550000,68764530,STK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2694,267810,KR7267810000,앙츠,KONEX,일반기업부,1093,0,0,0.00,0,0,0,0,0,1953191000,1787000,KNX
2695,215050,KR7215050006,비엔디생활건강,KONEX,일반기업부,912,0,0,0.00,0,0,0,0,0,1934959392,2121666,KNX
2696,267060,KR7267060002,명진홀딩스,KONEX,일반기업부,181,0,0,0.00,0,0,0,0,0,1654666524,9141804,KNX
2697,308700,KR7308700004,테크엔,KONEX,일반기업부,405,1,18,4.65,445,445,347,102,43292,1620000000,4000000,KNX


In [ ]:
# 2022.12.22 펜오션 

In [3]:
STOCK_CODE = '028670'
stock_price = fdr.DataReader(STOCK_CODE, '2022.12','2023.1')

In [4]:
temp_stock_price = stock_price.reset_index()

In [5]:
temp_stock_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    21 non-null     datetime64[ns]
 1   Open    21 non-null     int64         
 2   High    21 non-null     int64         
 3   Low     21 non-null     int64         
 4   Close   21 non-null     int64         
 5   Volume  21 non-null     int64         
 6   Change  21 non-null     float64       
dtypes: datetime64[ns](1), float64(1), int64(5)
memory usage: 1.3 KB


In [17]:
pre_price = temp_stock_price.query('Date <= 20221222').iloc[-8:-1,:].filter(items =['Date','Close'])
target_price = temp_stock_price.query('Date == 20221222').filter(items = ['Date','Open'])

In [18]:
pre_price = pre_price.rename(columns={'Close': 'Price'})
target_price = target_price.rename(columns={'Open': 'Price'})

In [19]:
df = pd.concat([pre_price,target_price]).reset_index(drop=True)

In [20]:
df['change'] = df['Price'].pct_change()

In [22]:
df.dropna()

,Date,Price,change
1,2022-12-14,5640,-0.001770
2,2022-12-15,5590,-0.008865
3,2022-12-16,5880,0.051878
4,2022-12-19,5880,0.000000
5,2022-12-20,5690,-0.032313
6,2022-12-21,5800,0.019332
7,2022-12-22,5920,0.020690


In [11]:
#한국투자 API 보안 인증키 발급 
APP_KEY = "PSDbkNNrIX3j60cW2MEFnBGZDZ1BzpgDxvC0"
APP_SECRET = "QfaZRX24K15ojO4yjG5o5dM+SUvJT4PYiK+tKKzgikyhmfrnTXoxOwn9rCf2hzXdtOhh4v3/O4bSFfQgS+u6ojdb6KgWuyub4GRraoWYaJTJyHSRqWUPab2GiCvrI/aIjx5S925MohrETq0k/eopFHgMInG0lmZdAh5TaegaZ1NDpNvPW8Y="
URL_BASE = "https://openapi.koreainvestment.com:9443"
PATH = "oauth2/tokenP"
URL = f"{URL_BASE}/{PATH}"

headers = {"content-type":"application/json"}
body = {"grant_type":"client_credentials",
        "appkey":APP_KEY, 
        "appsecret":APP_SECRET}

res = requests.post(URL, headers=headers, data=json.dumps(body))
ACCESS_TOKEN = res.json()["access_token"]

In [ ]:
ACCESS_TOKEN

In [12]:
json.dumps(body)

'{"grant_type": "client_credentials", "appkey": "PSDbkNNrIX3j60cW2MEFnBGZDZ1BzpgDxvC0", "appsecret": "QfaZRX24K15ojO4yjG5o5dM+SUvJT4PYiK+tKKzgikyhmfrnTXoxOwn9rCf2hzXdtOhh4v3/O4bSFfQgS+u6ojdb6KgWuyub4GRraoWYaJTJyHSRqWUPab2GiCvrI/aIjx5S925MohrETq0k/eopFHgMInG0lmZdAh5TaegaZ1NDpNvPW8Y="}'

In [26]:
#예제 삼성전자 주가 정보 받아오기
URL = 'https://openapi.koreainvestment.com:9443/uapi/domestic-stock/v1/quotations/inquire-price'

headers = {"Content-Type":"application/json", 
           "authorization": f"Bearer {ACCESS_TOKEN}",
           "appKey":APP_KEY,
           "appSecret":APP_SECRET,
           "tr_id":"FHKST01010100"}

params = {
    "fid_cond_mrkt_div_code":"J",
    "fid_input_iscd":"005930"
}

res = requests.get(URL, headers=headers, params=params)

In [35]:
#예제 삼성전자 주가 정보 받아오기 
URL = "https://openapi.koreainvestment.com:9443/uapi/domestic-stock/v1/quotations/search-info"

headers = {"Content-Type":"application/json", 
           "authorization": f"Bearer {ACCESS_TOKEN}",
           "appKey":APP_KEY,
           "appSecret":APP_SECRET,
           "tr_id":"FHKST01010100"}
params = {
    "fid_cond_mrkt_div_code":"J",
    "fid_input_iscd":"005930"
}

res = requests.get(URL, headers=headers, params=params)

In [37]:
res

<Response [200]>

In [49]:
#예제 삼성전자 기본정보 받아오기 
URL = 'https://openapi.koreainvestment.com:9443/uapi/domestic-stock/v1/quotations/inquire-price'

headers = {"Content-Type":"application/json", 
           "authorization": f"Bearer {ACCESS_TOKEN}",
           "appKey":APP_KEY,
           "appSecret":APP_SECRET,
           "tr_id":"CTPF1604R"}

params = {
    "PDNO":"005930",
    "PRDT_TYPE_CD":"300"
}

res = requests.get(URL, headers=headers, params=params)

In [63]:
'''코스피주식종목코드(kospi_code.mst) 정제 파이썬 파일'''

import urllib.request
import ssl
import zipfile
import os
import pandas as pd

base_dir = os.getcwd()

def kospi_master_download(base_dir, verbose=False):
    cwd = os.getcwd()
    if (verbose): print(f"current directory is {cwd}")
    ssl._create_default_https_context = ssl._create_unverified_context

    urllib.request.urlretrieve("https://new.real.download.dws.co.kr/common/master/kospi_code.mst.zip",
                               base_dir + "\\kospi_code.zip")

    os.chdir(base_dir)
    if (verbose): print(f"change directory to {base_dir}")
    kospi_zip = zipfile.ZipFile('kospi_code.zip')
    kospi_zip.extractall()

    kospi_zip.close()

    if os.path.exists("kospi_code.zip"):
        os.remove("kospi_code.zip")


def get_kospi_master_dataframe(base_dir):
    file_name = base_dir + "\\kospi_code.mst"
    tmp_fil1 = base_dir + "\\kospi_code_part1.tmp"
    tmp_fil2 = base_dir + "\\kospi_code_part2.tmp"

    wf1 = open(tmp_fil1, mode="w")
    wf2 = open(tmp_fil2, mode="w")

    with open(file_name, mode="r", encoding="cp949") as f:
        for row in f:
            rf1 = row[0:len(row) - 228]
            rf1_1 = rf1[0:9].rstrip()
            rf1_2 = rf1[9:21].rstrip()
            rf1_3 = rf1[21:].strip()
            wf1.write(rf1_1 + ',' + rf1_2 + ',' + rf1_3 + '\n')
            rf2 = row[-228:]
            wf2.write(rf2)

    wf1.close()
    wf2.close()

    part1_columns = ['단축코드', '표준코드', '한글명']
    df1 = pd.read_csv(tmp_fil1, header=None, names=part1_columns, encoding='UTF8')

    field_specs = [2, 1, 4, 4, 4,
                   1, 1, 1, 1, 1,
                   1, 1, 1, 1, 1,
                   1, 1, 1, 1, 1,
                   1, 1, 1, 1, 1,
                   1, 1, 1, 1, 1,
                   1, 9, 5, 5, 1,
                   1, 1, 2, 1, 1,
                   1, 2, 2, 2, 3,
                   1, 3, 12, 12, 8,
                   15, 21, 2, 7, 1,
                   1, 1, 1, 1, 9,
                   9, 9, 5, 9, 8,
                   9, 3, 1, 1, 1
                   ]

    part2_columns = ['그룹코드', '시가총액규모', '지수업종대분류', '지수업종중분류', '지수업종소분류',
                     '제조업', '저유동성', '지배구조지수종목', 'KOSPI200섹터업종', 'KOSPI100',
                     'KOSPI50', 'KRX', 'ETP', 'ELW발행', 'KRX100',
                     'KRX자동차', 'KRX반도체', 'KRX바이오', 'KRX은행', 'SPAC',
                     'KRX에너지화학', 'KRX철강', '단기과열', 'KRX미디어통신', 'KRX건설',
                     'Non1', 'KRX증권', 'KRX선박', 'KRX섹터_보험', 'KRX섹터_운송',
                     'SRI', '기준가', '매매수량단위', '시간외수량단위', '거래정지',
                     '정리매매', '관리종목', '시장경고', '경고예고', '불성실공시',
                     '우회상장', '락구분', '액면변경', '증자구분', '증거금비율',
                     '신용가능', '신용기간', '전일거래량', '액면가', '상장일자',
                     '상장주수', '자본금', '결산월', '공모가', '우선주',
                     '공매도과열', '이상급등', 'KRX300', 'KOSPI', '매출액',
                     '영업이익', '경상이익', '당기순이익', 'ROE', '기준년월',
                     '시가총액', '그룹사코드', '회사신용한도초과', '담보대출가능', '대주가능'
                     ]

    df2 = pd.read_fwf(tmp_fil2, widths=field_specs, names=part2_columns)

    df = pd.merge(df1, df2, how='outer', left_index=True, right_index=True)

    # clean temporary file and dataframe
    del (df1)
    del (df2)
    os.remove(tmp_fil1)
    os.remove(tmp_fil2)
    
    print("Done")

    return df


kospi_master_download(base_dir)
df = get_kospi_master_dataframe(base_dir) 

#df3 = df[df['KRX증권'] == 'Y']
df3 = df
# print(df3[['단축코드', '한글명', 'KRX', 'KRX증권', '기준가', '증거금비율', '상장일자', 'ROE']])
df3.to_csv('kospi_code.csv',index=False, encoding='utf-8-sig')# 현재 위치에 엑셀파일로 저장


Done


In [65]:
Kospi_df = pd.read_csv('./kospi_code.csv')

In [67]:
'''코스닥주식종목코드(kosdaq_code.mst) 정제 파이썬 파일'''

import pandas as pd
import urllib.request
import ssl
import zipfile
import os

base_dir = os.getcwd()

def kosdaq_master_download(base_dir, verbose=False):

    cwd = os.getcwd()
    if (verbose): print(f"current directory is {cwd}")
    ssl._create_default_https_context = ssl._create_unverified_context
    
    urllib.request.urlretrieve("https://new.real.download.dws.co.kr/common/master/kosdaq_code.mst.zip",
                               base_dir + "\\kosdaq_code.zip")

    os.chdir(base_dir)
    if (verbose): print(f"change directory to {base_dir}")
    kosdaq_zip = zipfile.ZipFile('kosdaq_code.zip')
    kosdaq_zip.extractall()
    
    kosdaq_zip.close()

    if os.path.exists("kosdaq_code.zip"):
        os.remove("kosdaq_code.zip")

def get_kosdaq_master_dataframe(base_dir):
    file_name = base_dir + "\\kosdaq_code.mst"
    tmp_fil1 = base_dir + "\\kosdaq_code_part1.tmp"
    tmp_fil2 = base_dir + "\\kosdaq_code_part2.tmp"

    wf1 = open(tmp_fil1, mode="w")
    wf2 = open(tmp_fil2, mode="w")

    with open(file_name, mode="r", encoding="cp949") as f:
        for row in f:
            rf1 = row[0:len(row) - 222]
            rf1_1 = rf1[0:9].rstrip()
            rf1_2 = rf1[9:21].rstrip()
            rf1_3 = rf1[21:].strip()
            wf1.write(rf1_1 + ',' + rf1_2 + ',' + rf1_3 + '\n')
            rf2 = row[-222:]
            wf2.write(rf2)

    wf1.close()
    wf2.close()

    part1_columns = ['단축코드','표준코드','한글종목명']
    df1 = pd.read_csv(tmp_fil1, header=None, names=part1_columns, encoding='UTF8')

    field_specs = [2, 1,
                   4, 4, 4, 1, 1,
                   1, 1, 1, 1, 1,
                   1, 1, 1, 1, 1,
                   1, 1, 1, 1, 1,
                   1, 1, 1, 1, 9,
                   5, 5, 1, 1, 1,
                   2, 1, 1, 1, 2,
                   2, 2, 3, 1, 3,
                   12, 12, 8, 15, 21,
                   2, 7, 1, 1, 1,
                   1, 9, 9, 9, 5,
                   9, 8, 9, 3, 1,
                   1, 1
                   ]

    part2_columns = ['증권그룹구분코드','시가총액 규모 구분 코드 유가',
                     '지수업종 대분류 코드','지수 업종 중분류 코드','지수업종 소분류 코드','벤처기업 여부 (Y/N)',
                     '저유동성종목 여부','KRX 종목 여부','ETP 상품구분코드','KRX100 종목 여부 (Y/N)',
                     'KRX 자동차 여부','KRX 반도체 여부','KRX 바이오 여부','KRX 은행 여부','기업인수목적회사여부',
                     'KRX 에너지 화학 여부','KRX 철강 여부','단기과열종목구분코드','KRX 미디어 통신 여부',
                     'KRX 건설 여부','(코스닥)투자주의환기종목여부','KRX 증권 구분','KRX 선박 구분',
                     'KRX섹터지수 보험여부','KRX섹터지수 운송여부','KOSDAQ150지수여부 (Y,N)','주식 기준가',
                     '정규 시장 매매 수량 단위','시간외 시장 매매 수량 단위','거래정지 여부','정리매매 여부',
                     '관리 종목 여부','시장 경고 구분 코드','시장 경고위험 예고 여부','불성실 공시 여부',
                     '우회 상장 여부','락구분 코드','액면가 변경 구분 코드','증자 구분 코드','증거금 비율',
                     '신용주문 가능 여부','신용기간','전일 거래량','주식 액면가','주식 상장 일자','상장 주수(천)',
                     '자본금','결산 월','공모 가격','우선주 구분 코드','공매도과열종목여부','이상급등종목여부',
                     'KRX300 종목 여부 (Y/N)','매출액','영업이익','경상이익','단기순이익','ROE(자기자본이익률)',
                     '기준년월','전일기준 시가총액 (억)','그룹사 코드','회사신용한도초과여부','담보대출가능여부','대주가능여부'
                     ]

    df2 = pd.read_fwf(tmp_fil2, widths=field_specs, names=part2_columns)

    df = pd.merge(df1, df2, how='outer', left_index=True, right_index=True)

    # clean temporary file and dataframe
    del (df1)
    del (df2)
    os.remove(tmp_fil1)
    os.remove(tmp_fil2)

    print("Done")

    return df

kosdaq_master_download(base_dir)
df = get_kosdaq_master_dataframe(base_dir)

df.to_csv('kosdaq_code.csv',index=False, encoding='utf-8-sig')  # 현재 위치에 엑셀파일로 저장


Done


In [69]:
Kosdaq_df = pd.read_csv('./kosdaq_code.csv')

In [75]:
kosdaq = Kosdaq_df.filter(items= ['단축코드','한글종목명'])

In [79]:
kospi = Kospi_df.filter(items= ['단축코드','한글명'])

In [82]:
Kospi_df = pd.read_csv('./kospi_code.csv')
Kosdaq_df = pd.read_csv('./kosdaq_code.csv')
kospi = Kospi_df.filter(items= ['단축코드','한글명'])
kosdaq = Kosdaq_df.filter(items= ['단축코드','한글종목명'])

,단축코드,한글명
377,005930,삼성전자


In [86]:
#예제 시간외일자별 주가
URL = 'https://openapi.koreainvestment.com:9443/uapi/domestic-stock/v1/quotations/inquire-daily-overtimeprice'

headers = {"Content-Type":"application/json", 
           "authorization": f"Bearer {ACCESS_TOKEN}",
           "appKey":APP_KEY,
           "appSecret":APP_SECRET,
           "tr_id":"FHPST02320000",
           "tr_cont":"N"}

params = {
    "FID_COND_MRKT_DIV_CODE":"J",
    "FID_INPUT_ISCD":"005930"
}

res = requests.get(URL, headers=headers, params=params)

In [87]:
res.jso

{'output1': {'ovtm_untp_prpr': '62900',
  'ovtm_untp_prdy_vrss': '100',
  'ovtm_untp_prdy_vrss_sign': '2',
  'ovtm_untp_prdy_ctrt': '0.16',
  'ovtm_untp_vol': '38758',
  'ovtm_untp_tr_pbmn': '2433433200',
  'ovtm_untp_mxpr': '69000',
  'ovtm_untp_llam': '56600',
  'ovtm_untp_oprc': '62700',
  'ovtm_untp_hgpr': '62900',
  'ovtm_untp_lwpr': '62600',
  'ovtm_untp_antc_cnpr': '0',
  'ovtm_untp_antc_cntg_vrss': '0',
  'ovtm_untp_antc_cntg_vrss_sign': '3',
  'ovtm_untp_antc_cntg_ctrt': '0.00',
  'ovtm_untp_antc_vol': '0'},
 'output2': [{'stck_bsop_date': '20230210',
   'ovtm_untp_prpr': '62900',
   'ovtm_untp_prdy_vrss': '100',
   'ovtm_untp_prdy_vrss_sign': '2',
   'ovtm_untp_prdy_ctrt': '0.16',
   'ovtm_untp_vol': '38758',
   'stck_clpr': '62800',
   'prdy_vrss': '-200',
   'prdy_vrss_sign': '5',
   'prdy_ctrt': '-0.32',
   'acml_vol': '9382576',
   'ovtm_untp_tr_pbmn': '2433433200'},
  {'stck_bsop_date': '20230209',
   'ovtm_untp_prpr': '63300',
   'ovtm_untp_prdy_vrss': '300',
   'ovtm_u